In [ ]:
import sys
import argparse
import glob
import cv2
import os
import numpy as np
import torch
import time
from tqdm import tqdm
from pathlib import Path

from droid_slam.droid import Droid
import droid_backends

from lac.utils.plotting import plot_path_3d, plot_3d_points, plot_poses
from lac.util import load_data
from lac.params import LAC_BASE_PATH

%load_ext autoreload
%autoreload 2

# Load data


In [ ]:
def image_stream(
    datapath,
    image_size=[360, 640],
    # image_size=[270, 480],
    intrinsics_vec=[914.0152, 914.0152, 640.0, 360.0],
    stereo=False,
    start_frame=0,
    stride=1,
):
    """image generator"""

    # read all png images in folder
    ht0, wd0 = [720, 1280]
    images_left = sorted(glob.glob(os.path.join(datapath, "FrontLeft/*.png")))
    images_right = sorted(glob.glob(os.path.join(datapath, "FrontRight/*.png")))

    data = []
    # for t in range(start_frame, len(images_left), stride):
    for t in range(start_frame, 100, stride):
        images = [cv2.resize(cv2.imread(images_left[t]), (image_size[1], image_size[0]))]
        if stereo:
            images += [cv2.resize(cv2.imread(images_right[t]), (image_size[1], image_size[0]))]

        images = torch.from_numpy(np.stack(images, 0)).permute(0, 3, 1, 2)
        intrinsics = 0.5 * torch.as_tensor(intrinsics_vec)

        data.append((t, images, intrinsics))

    return data

In [ ]:
parser = argparse.ArgumentParser()
# parser.add_argument("--datapath", default="data/LAC")
parser.add_argument("--weights", default="/home/lac/opt/DROID-SLAM/droid.pth")
parser.add_argument("--buffer", type=int, default=2500)
parser.add_argument("--disable_vis", action="store_true")
# parser.add_argument("--plot_curve", action="store_true")
# parser.add_argument("--image_size", default=[270, 480])
parser.add_argument("--image_size", default=[360, 640])
# parser.add_argument("--id", type=int, default=-1)

parser.add_argument("--beta", type=float, default=0.3, help="weight for translation / rotation components of flow")
parser.add_argument(
    "--filter_thresh",
    type=float,
    default=2.4,
    help="how much motion before considering new keyframe",
)
parser.add_argument("--warmup", type=int, default=8, help="number of warmup frames")
parser.add_argument("--keyframe_thresh", type=float, default=4.0, help="threshold to create a new keyframe")
parser.add_argument(
    "--frontend_thresh",
    type=float,
    default=16.0,
    help="add edges between frames whithin this distance",
)
parser.add_argument("--frontend_window", type=int, default=25, help="frontend optimization window")
parser.add_argument("--frontend_radius", type=int, default=2, help="force edges between frames within radius")
parser.add_argument("--frontend_nms", type=int, default=1, help="non-maximal supression of edges")

parser.add_argument("--backend_thresh", type=float, default=20.0)
parser.add_argument("--backend_radius", type=int, default=2)
parser.add_argument("--backend_nms", type=int, default=3)
parser.add_argument("--upsample", action="store_true")

In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/DataCollectionAgent/full_spiral_map1_preset0"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

In [ ]:
args = parser.parse_args([])
args.stereo = True

In [ ]:
stream = image_stream(str(data_path), stereo=True, start_frame=50, stride=2)

# Tracking


In [ ]:
BA_RATE = 1000
START_FRAME = 50

torch.cuda.empty_cache()
droid = Droid(args)

start_time = time.time()

for tstamp, image, intrinsics in tqdm(stream):
    if tstamp < START_FRAME:
        continue
    droid.track(tstamp, image, intrinsics=intrinsics)

    # if tstamp % BA_RATE == 0:
    #     droid.backend(7)

print("Tracking ran {} frames in {} seconds".format(len(stream), time.time() - start_time))

In [ ]:
traj_est = droid.terminate(stream)
np.save("droid_traj.npy", traj_est)

# Inspect output and warping


In [ ]:
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
from scipy.spatial.transform import Rotation

from lac.utils.plotting import plot_path_3d, plot_3d_points, plot_poses
from lac.utils.frames import OPENCV_TO_CAMERA_PASSIVE, opencv_to_camera
from lac.util import load_data

In [ ]:
data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset0"
# data_path = "/home/shared/data_raw/LAC/runs/full_spiral_map1_preset1_recovery_agent"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

# traj_est = np.load("/home/lac/LunarAutonomyChallenge/data/droid_trajs/droid_traj_partial.npy")
droid_result = np.load(Path(data_path) / "droid.npz")
traj_est = droid_result["trajectory"]

START_FRAME = 80

In [ ]:
FLU_TO_OPENGL = np.array([[0, 0, -1], [1, 0, 0], [0, -1, 0]])

camera_poses = []
droid_poses = []
ns_poses = []

for vec in traj_est:
    t = vec[:3]
    q = vec[3:]
    R = Rotation.from_quat(q).as_matrix()
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t
    droid_poses.append(T)

    transf = np.eye(4)
    transf[:3, :3] = OPENCV_TO_CAMERA_PASSIVE.T
    T = transf @ T
    T[:3, :3] = T[:3, :3] @ OPENCV_TO_CAMERA_PASSIVE
    camera_poses.append(T.copy())
    T[:3, :3] = T[:3, :3] @ FLU_TO_OPENGL
    ns_poses.append(T.copy())

In [ ]:
aligned_poses = [initial_pose @ p for p in camera_poses]

fig = go.Figure()
fig = plot_poses(poses[100:21350], fig=fig, no_axes=True, name="Ground Truth", color="black")
fig = plot_poses(aligned_poses[:-1], fig=fig, no_axes=True, name="Droid")
# fig = plot_poses(droid_poses[:1000], fig=fig, no_axes=False, name="Droid (aligned)", color="blue")
# ns_poses_ds = ns_poses[::5]
# fig = plot_poses(ns_poses_ds[:200], fig=fig, no_axes=False, name="Droid (aligned)", color="blue")
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()

In [ ]:
# Use the first 10 poses (40 frames) to estimate scale
scale = np.linalg.norm(aligned_poses[0][:3, 3] - aligned_poses[10][:3, 3]) / np.linalg.norm(
    poses[START_FRAME][:3, 3] - poses[START_FRAME + 4 * 10][:3, 3]
)
print("Scale: ", scale)

# Reconstruction


In [ ]:
from lietorch import SE3

In [ ]:
poses = droid.video.poses
disps = droid.video.disps


points = droid_backends.iproj(SE3(poses).inv().data, disps, droid.video.intrinsics[0]).cpu()

In [ ]:
points.shape

In [ ]:
pts = points.reshape(-1, 3).cpu().numpy()

In [ ]:
pts.shape

In [ ]:
import open3d as o3d
import numpy as np

In [ ]:
t = droid.video.counter.value
tstamps = droid.video.tstamp[:t].cpu().numpy()
images = droid.video.images[:t].cpu().numpy()
disps = droid.video.disps_up[:t].cpu().numpy()
poses = droid.video.poses[:t].cpu().numpy()
intrinsics = droid.video.intrinsics[:t].cpu().numpy()

In [ ]:
# depth scale factor
depth_scale = 1 / 256

point_cloud = o3d.geometry.PointCloud()

u, v = np.meshgrid(range(images.shape[3]), range(images.shape[2]))
x = (u - intrinsics[0, 2]) * disps / intrinsics[0, 0] * depth_scale
y = (v - intrinsics[1, 2]) * disps / intrinsics[1, 1] * depth_scale
z = disps * depth_scale

points = np.vstack((x.flatten(), y.flatten(), z.flatten(), np.ones_like(x.flatten()))).T

colors = images[:, [2, 1, 0], :, :].transpose(0, 2, 3, 1).reshape(-1, 3) / 255

point_cloud.points = o3d.utility.Vector3dVector(points[:, :3])
point_cloud.colors = o3d.utility.Vector3dVector(colors)


for i in range(poses.shape[0]):
    # homogeneous transform matrix로 변환
    pose_matrix = np.eye(4)
    pose_matrix[:3, :3] = Rotation.from_quat(poses[i, 3:]).as_matrix()
    pose_matrix[:3, 3] = poses[i, :3]

    # point_cloud 객체의 위치 및 방향 설정
    point_cloud.transform(pose_matrix)


o3d.visualization.draw_geometries([point_cloud])

In [ ]:
point_cloud

In [ ]:
plot_3d_points(pts[::100])

# Inspect splat slam output


In [ ]:
data_path = Path(LAC_BASE_PATH) / "output/LocalizationAgent/map1_preset0_4m_spiral"
initial_pose, lander_pose, poses, imu_data, cam_config = load_data(data_path)

output_path = Path("/home/lac/Splat-SLAM/output")
video = np.load(output_path / "video_final.npz")

In [ ]:
slam_poses = video["poses"]

In [ ]:
fig = plot_poses(poses, no_axes=True, name="Ground Truth", color="black")
fig = plot_poses(slam_poses, fig=fig, no_axes=True, name="Splat-SLAM")
fig.update_layout(width=1600, height=900, scene_aspectmode="data")
fig.show()